In [1]:
import sqlite3

# **Storing structured data**

In [2]:
# creation a connection
conn = sqlite3.connect('test.db')

In [3]:
# create cursor object to exceute SQL commands
cursor = conn.cursor()

In [4]:
# create a table
cursor.execute('''CREATE TABLE IF NOT EXISTS employees (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  age INTEGER,
  department TEXT
  )''')

In [5]:
# insert data into table
cursor.execute("INSERT INTO employees (name, age, department) VALUES ('John', 30, 'IT')")

In [6]:
cursor.execute("SELECT * FROM employees")

In [7]:
rows = cursor.fetchall()

In [8]:
rows

[(1, 'John', 30, 'IT')]

In [9]:
conn.commit() # commit

In [10]:
conn.close() # close the connection

# **Storing and Retrieving Vector Data in SQLite**

In [11]:
import numpy as np

In [12]:
conn = sqlite3.connect('vector-db.db')
cursor = conn.cursor()

In [14]:
cursor.execute('''CREATE TABLE IF NOT EXISTS vectors (
  id INTEGER PRIMARY KEY,
  vector BLOB NOT NULL
  )''')

In [15]:
vect1 = np.array([1.2,3.4,2.1,0.8])
vect2 = np.array([2.7,1.5,3.9,2.3])
vect1.tobytes()

b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?'

In [16]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect1.tobytes()), ))

In [17]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect2.tobytes()), ))

In [19]:
cursor.execute("SELECT vector FROM vectors")

In [20]:
rows = cursor.fetchall()
rows

[(b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)]

In [21]:
vector = np.frombuffer(rows[0][0], dtype=np.float64)

In [22]:
vector

array([1.2, 3.4, 2.1, 0.8])

In [23]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[0], dtype=np.float64)
    vectors.append(vector)

In [24]:
vectors

[array([1.2, 3.4, 2.1, 0.8]), array([2.7, 1.5, 3.9, 2.3])]

# **Vector Similarity Search (VSS)**

In [26]:
query_vect = np.array([1.0, 3.2, 2.0, 0.5])

In [27]:
cursor.execute("""SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC
""",(sqlite3.Binary(query_vect.tobytes()),))

In [28]:
res = cursor.fetchone() #finding the top one

In [30]:
np.frombuffer(res[0],dtype=np.float64)

array([2.7, 1.5, 3.9, 2.3])